In [14]:
!pip3 install sentencepiece

   ---------------------------------------- 0.0/1.1 MB ? eta -:--:--
   ---------------------------------------- 1.1/1.1 MB 9.2 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [20]:
import torch
from tqdm import tqdm
from transformers import pipeline
import pandas as pd
from torch.nn.utils.rnn import pad_sequence
from torch import nn, optim
from torch.utils.data import DataLoader, TensorDataset
import sentencepiece as spm

P_df = pd.read_csv("C:\\Emotion Classification\\treeModelsData\\positive_emotions.csv")
N_df = pd.read_csv("C:\\Emotion Classification\\treeModelsData\\negative_emotions.csv")
TOKENIZER = 'C:\Emotion Classification\\traning\\vocab\\tokenizer.model'

sp = spm.SentencePieceProcessor()
sp.load(TOKENIZER)

<>:12: SyntaxWarning: invalid escape sequence '\E'
<>:12: SyntaxWarning: invalid escape sequence '\E'
C:\Users\illya\AppData\Local\Temp\ipykernel_26308\448116470.py:12: SyntaxWarning: invalid escape sequence '\E'
  TOKENIZER = 'C:\Emotion Classification\\traning\\vocab\\tokenizer.model'


True

In [21]:
P_df.shape[0] + N_df.shape[0], P_df.shape[1]

(43332, 26)

In [22]:
P_df.dtypes

text              object
admiration         int64
amusement          int64
anger              int64
annoyance          int64
approval           int64
caring             int64
confusion          int64
curiosity          int64
desire             int64
disappointment     int64
disapproval        int64
disgust            int64
embarrassment      int64
excitement         int64
fear               int64
gratitude          int64
grief              int64
joy                int64
love               int64
nervousness        int64
optimism           int64
pride              int64
relief             int64
remorse            int64
sadness            int64
dtype: object

In [23]:
split_index = int(P_df.shape[0] * 0.8)
split_index

21153

In [24]:
def pipeline_text(data):
    data = data.copy()
    
    data["text"] = data["text"].apply(lambda x: sp.encode(x, out_type=int))
    y = data.drop(columns=['text'])
    
    list_of_lists = data['text'].tolist()
    tensor_list = [torch.tensor(seq, dtype=torch.long) for seq in list_of_lists]
    X = pad_sequence(tensor_list, batch_first=True, padding_value=0)
    y = torch.tensor(y.values, dtype=torch.long)
    
    X_train = X[:split_index]
    X_test = X[split_index:]
    
    y_train = y[:split_index]
    y_test = y[split_index:]
    
    
    return X_train, y_train, X_test, y_test
    
X_train, y_train, X_test, y_test = pipeline_text(P_df)

In [25]:
P_ds = TensorDataset()